In [1]:
import requests
import json
from loguru import logger
from pystac.item_collection import ItemCollection

namespace = "alice"

ogc_api_endpoint = f"http://zoo-project-dru-service/{namespace}/ogc-api/processes"

r = requests.get(ogc_api_endpoint)

r.json()


{'processes': [{'id': 'echo',
   'title': 'Echo input',
   'description': 'Simply echo the value provided as input',
   'mutable': False,
   'version': '2.0.0',
   'metadata': [{'title': 'Demo'}],
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'self',
     'type': 'application/json',
     'title': 'Process Description',
     'href': 'http://localhost:8080/alice/ogc-api/processes/echo'}]},
  {'id': 'water_bodies',
   'title': 'Water bodies detection based on NDWI and otsu threshold',
   'description': 'Water bodies detection based on NDWI and otsu threshold',
   'mutable': True,
   'version': '1.4.1',
   'metadata': [{'role': 'https://schema.org/softwareVersion',
     'value': '1.4.1'}],
   'outputTransmission': ['value', 'reference'],
   'jobControlOptions': ['async-execute', 'dismiss'],
   'links': [{'rel': 'self',
     'type': 'application/json',
     'title': 'Process Description',
     'hre

In [2]:
r = requests.delete(f"{ogc_api_endpoint}/water_bodies")
r.status_code

204

In [3]:
headers = {}
response = requests.post(
    f"{ogc_api_endpoint}",
    headers = dict([("Content-Type", "application/json")], **headers),
    json = {
      "executionUnit": {
        "href": "https://github.com/Terradue/ogc-eo-application-package-hands-on/releases/download/1.5.0/app-water-bodies-cloud-native.1.5.0.cwl",
        "type": "application/cwl"
      }
    }
)

In [4]:
print(response.json())

{'id': 'water_bodies', 'title': 'Water bodies detection based on NDWI and otsu threshold', 'description': 'Water bodies detection based on NDWI and otsu threshold', 'mutable': True, 'version': '1.4.1', 'metadata': [{'role': 'https://schema.org/softwareVersion', 'value': '1.4.1'}], 'outputTransmission': ['value', 'reference'], 'jobControlOptions': ['async-execute', 'dismiss'], 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute', 'type': 'application/json', 'title': 'Execute End Point', 'href': 'http://localhost:8080/alice/ogc-api/processes/water_bodies/execution'}]}


In [5]:
print(response.status_code)

201


In [6]:
print(response.text)

{"id":"water_bodies","title":"Water bodies detection based on NDWI and otsu threshold","description":"Water bodies detection based on NDWI and otsu threshold","mutable":true,"version":"1.4.1","metadata":[{"role":"https://schema.org/softwareVersion","value":"1.4.1"}],"outputTransmission":["value","reference"],"jobControlOptions":["async-execute","dismiss"],"links":[{"rel":"http://www.opengis.net/def/rel/ogc/1.0/execute","type":"application/json","title":"Execute End Point","href":"http://localhost:8080/alice/ogc-api/processes/water_bodies/execution"}]}



In [ ]:
ogc_api_endpoint = "http://zoo-project-dru-service/alice/ogc-api/processes"

r = requests.get(f"{ogc_api_endpoint}/water_bodies")

r.json()

In [7]:
headers = {
    'accept': '*/*',
    'Prefer': 'respond-async;return=representation',
    'Content-Type': 'application/json'
}
data = {
    "inputs": {
        "stac_items": [
            "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A",
            "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A"
        ],
        "aoi": "-121.399,39.834,-120.74,40.472",
        "epsg": "EPSG:4326",
        "bands": [
            "green",
            "nir"
        ]
    }
}

response = requests.post(f"{ogc_api_endpoint}/water_bodies/execution", headers=headers, json=data)

print(response.status_code)
print(response.json())

201
{'jobID': 'c6f823a2-8c57-11ef-9c0f-0ac1619c2c37', 'type': 'process', 'processID': 'water_bodies', 'created': '2024-10-17T07:16:50.299Z', 'started': '2024-10-17T07:16:50.299Z', 'updated': '2024-10-17T07:16:50.299Z', 'status': 'running', 'message': 'ZOO-Kernel accepted to run your service!', 'links': [{'title': 'Status location', 'rel': 'monitor', 'type': 'application/json', 'href': 'http://localhost:8080/alice/ogc-api/jobs/c6f823a2-8c57-11ef-9c0f-0ac1619c2c37'}]}


In [8]:
jobid = response.json().get("jobID")

In [9]:
import time
job_url = 'http://zoo-project-dru-service/alice' + '/ogc-api/jobs/' + jobid

headers = {
    'accept': 'application/json',
}

while True:
    response = requests.get(job_url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        status = data.get("status")
        if status == "successful":
            print("Job status:", status)
            break  # Exit the loop if OK
        else:
            print("Job status:", status)
            time.sleep(10)
    else:
        print("Error:", response.status_code)
        break  

Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: successful


In [10]:
job_results = 'http://zoo-project-dru-service/alice' + '/ogc-api/jobs/' + jobid + "/results"

headers = {
    'accept': 'application/json',
}
response = requests.get(job_results, headers=headers)

print(response.json())

{'stac_catalog': {'value': {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'stac_version': '1.0.0', 'id': 'S2B_10TFK_20210713_0_L2A', 'properties': {'proj:epsg': 32610, 'proj:geometry': {'type': 'Polygon', 'coordinates': [[[636990.0, 4410550.0], [691590.0, 4410550.0], [691590.0, 4482600.0], [636990.0, 4482600.0], [636990.0, 4410550.0]]]}, 'proj:bbox': [636990.0, 4410550.0, 691590.0, 4482600.0], 'proj:shape': [7205, 5460], 'proj:transform': [10.0, 0.0, 636990.0, 0.0, -10.0, 4482600.0, 0.0, 0.0, 1.0], 'proj:projjson': {'$schema': 'https://proj.org/schemas/v0.4/projjson.schema.json', 'type': 'ProjectedCRS', 'name': 'WGS 84 / UTM zone 10N', 'base_crs': {'name': 'WGS 84', 'datum': {'type': 'GeodeticReferenceFrame', 'name': 'World Geodetic System 1984', 'ellipsoid': {'name': 'WGS 84', 'semi_major_axis': 6378137, 'inverse_flattening': 298.257223563}}, 'coordinate_system': {'subtype': 'ellipsoidal', 'axis': [{'name': 'Geodetic latitude', 'abbreviation': 'Lat', 'direction': 'nort

In [11]:
response.json()

{'stac_catalog': {'value': {'type': 'FeatureCollection',
   'features': [{'type': 'Feature',
     'stac_version': '1.0.0',
     'id': 'S2B_10TFK_20210713_0_L2A',
     'properties': {'proj:epsg': 32610,
      'proj:geometry': {'type': 'Polygon',
       'coordinates': [[[636990.0, 4410550.0],
         [691590.0, 4410550.0],
         [691590.0, 4482600.0],
         [636990.0, 4482600.0],
         [636990.0, 4410550.0]]]},
      'proj:bbox': [636990.0, 4410550.0, 691590.0, 4482600.0],
      'proj:shape': [7205, 5460],
      'proj:transform': [10.0,
       0.0,
       636990.0,
       0.0,
       -10.0,
       4482600.0,
       0.0,
       0.0,
       1.0],
      'proj:projjson': {'$schema': 'https://proj.org/schemas/v0.4/projjson.schema.json',
       'type': 'ProjectedCRS',
       'name': 'WGS 84 / UTM zone 10N',
       'base_crs': {'name': 'WGS 84',
        'datum': {'type': 'GeodeticReferenceFrame',
         'name': 'World Geodetic System 1984',
         'ellipsoid': {'name': 'WGS 84',
 

In [12]:
ItemCollection.from_dict(response.json()["stac_catalog"]["value"]).items

[<Item id=S2B_10TFK_20210713_0_L2A>, <Item id=S2A_10TFK_20220524_0_L2A>]

In [13]:
for item in ItemCollection.from_dict(response.json()["stac_catalog"]["value"]).items:

    print(item.get_assets())
    print(json.dumps(item.get_assets()["data"].to_dict(), sort_keys=True, indent=4))

{'data': <Asset href=s3://results/c6f823a2-8c57-11ef-9c0f-0ac1619c2c37/c6f823a2-8c57-11ef-9c0f-0ac1619c2c37/S2B_10TFK_20210713_0_L2A/otsu.tif>}
{
    "href": "s3://results/c6f823a2-8c57-11ef-9c0f-0ac1619c2c37/c6f823a2-8c57-11ef-9c0f-0ac1619c2c37/S2B_10TFK_20210713_0_L2A/otsu.tif",
    "roles": [
        "data",
        "visual"
    ],
    "storage:endpoint": "http://eoap-zoo-project-localstack.eoap-zoo-project.svc.cluster.local:4566",
    "storage:platform": "eoap",
    "storage:region": "us-east-1",
    "storage:requester_pays": false,
    "storage:tier": "Standard",
    "type": "image/tiff; application=geotiff"
}
{'data': <Asset href=s3://results/c6f823a2-8c57-11ef-9c0f-0ac1619c2c37/c6f823a2-8c57-11ef-9c0f-0ac1619c2c37/S2A_10TFK_20220524_0_L2A/otsu.tif>}
{
    "href": "s3://results/c6f823a2-8c57-11ef-9c0f-0ac1619c2c37/c6f823a2-8c57-11ef-9c0f-0ac1619c2c37/S2A_10TFK_20220524_0_L2A/otsu.tif",
    "roles": [
        "data",
        "visual"
    ],
    "storage:endpoint": "http://eoap-zoo

In [14]:
col = ItemCollection.from_dict(response.json()["stac_catalog"]["value"])

In [130]:
col.items[0].links

[<Link rel=collection target=s3://results/9227b784-8bc9-11ef-af02-0ac1619c2c37/9227b784-8bc9-11ef-af02-0ac1619c2c37/collection.json>,
 <Link rel=root target=s3://results/9227b784-8bc9-11ef-af02-0ac1619c2c37/catalog.json>,
 <Link rel=self target=s3://results/9227b784-8bc9-11ef-af02-0ac1619c2c37/9227b784-8bc9-11ef-af02-0ac1619c2c37/S2B_10TFK_20210713_0_L2A/S2B_10TFK_20210713_0_L2A.json>,
 <Link rel=parent target=s3://results/9227b784-8bc9-11ef-af02-0ac1619c2c37/9227b784-8bc9-11ef-af02-0ac1619c2c37/collection.json>]

{'id': 'water_bodies',
 'title': 'Water bodies detection based on NDWI and otsu threshold',
 'description': 'Water bodies detection based on NDWI and otsu threshold',
 'mutable': True,
 'version': '1.4.1',
 'metadata': [{'role': 'https://schema.org/softwareVersion',
   'value': '1.4.1'}],
 'outputTransmission': ['value', 'reference'],
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'http://localhost:8080/alice/ogc-api/processes/water_bodies/execution'}],
 'inputs': {'aoi': {'title': 'area of interest',
   'description': 'area of interest as a bounding box',
   'schema': {'type': 'string'}},
  'bands': {'title': 'bands used for the NDWI',
   'description': 'bands used for the NDWI',
   'schema': {'type': 'string', 'default': "['green', 'nir']"}},
  'epsg': {'title': 'EPSG code',
   'description': 'EPSG code',
   'schema': {'type': 'string', 